In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
import igraph as ig
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn import Parameter
import torch.optim as optim
import math
import time
sns.set_theme(style="whitegrid", palette="pastel")

C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Dataset

In [2]:
data_list = []
flist = []
def import_node_db():
  global flist,data_list
  data_list = ["Stockholm","Saarbrucken","Freiburg"]
    
  floor_list1 = ["floor4_1","floor4_2","floor4_3","floor4_4","floor4_5","floor4_6","floor4_7","floor4_8",
                   "floor5_1","floor5_2","floor5_3","floor5_4","floor5_5","floor5_6","floor5_7","floor5_8",
                   "floor6_1","floor6_2","floor6_3","floor6_4","floor6_5","floor6_6","floor6_7","floor6_8","floor6_9",
                   "floor6_10","floor6_11","floor6_12","floor6_13","floor6_14","floor6_15","floor6_16","floor6_17","floor6_18",
                   "floor7_1","floor7_2","floor7_3","floor7_4","floor7_5","floor7_6","floor7_7","floor7_8"]

  floor_list2 = ["seq1_cloudy1","seq1_cloudy2","seq1_cloudy3","seq1_night1","seq1_night2","seq1_night3",
                 "seq2_cloudy1","seq2_cloudy2","seq2_cloudy3","seq2_night1","seq2_night2","seq2_night3",
                 "seq3_cloudy1","seq3_cloudy2","seq3_cloudy3","seq3_cloudy4","seq3_cloudy5","seq3_night1","seq3_night2","seq3_night3","seq3_sunny1","seq3_sunny2","seq3_sunny3",
                 "seq4_cloudy1","seq4_cloudy2","seq4_cloudy3","seq4_night1","seq4_night2","seq4_night3","seq4_sunny1","seq4_sunny2","seq4_sunny3"]

  floor_list3 = ["seq1_cloudy1","seq1_cloudy2","seq1_cloudy3","seq1_night1","seq1_night2","seq1_night3","seq1_sunny1","seq1_sunny2","seq1_sunny3","seq1_sunny4",
                 "seq2_cloudy1","seq2_cloudy2","seq2_cloudy3","seq2_night1","seq2_night2","seq2_night3","seq2_sunny1","seq2_sunny2","seq2_sunny3","seq2_sunny4",
                 "seq3_cloudy1","seq3_cloudy2","seq3_cloudy3","seq3_sunny1","seq3_sunny2","seq3_sunny3"]    

  flist = [floor_list1,floor_list2,floor_list3]
  # return flist
  node_df_list = []
  for i in range(3):
    df_list = []
    for j in flist[i]:
      if(i==0):
        p=6
      else:
        p=4
      node_df1 = pd.read_table("F:/prooo/dataset/{}/{}/nodes.dat".format(data_list[i],j),sep="\s+")
      # node_df1['floor']=j
      # node_df1['loc']=data_list[i]
      for k in range(8):
        node_df1["new_neighbor_ID_{}".format(k)] = [str(col) +"-"+ str(j) + "-" + str(data_list[i][:3])  for col in node_df1["neighbor_ID_{}".format(k)]]
      # node_df1['locfloor']=data_list[i]+"-"+j
      node_df1['node_id']=[str(col) +"-"+ str(j) + "-" + str(data_list[i][:3]) for col in node_df1['node_id']]
      node_df1['locfloor']=data_list[i]+"-"+j[:p]
      df_list.append(node_df1)
    node_df_list.append(pd.concat(df_list,ignore_index=True))
  main_list = pd.concat(node_df_list,ignore_index=True)
  return main_list
data = import_node_db()
data.head()

,node_id,placeholder,x,y,d1,d2,d3,d4,label,d5,...,view_num_7,new_neighbor_ID_0,new_neighbor_ID_1,new_neighbor_ID_2,new_neighbor_ID_3,new_neighbor_ID_4,new_neighbor_ID_5,new_neighbor_ID_6,new_neighbor_ID_7,locfloor
0,0-floor4_1-Sto,0,-1.000001,-0.200001,-1.025785,-0.097851,-1.000001,-0.200001,EV,1.205846e+09,...,0,1-floor4_1-Sto,2-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,Stockholm-floor4
1,1-floor4_1-Sto,0,-0.140001,0.259999,-0.303640,-0.002786,-0.200001,0.599999,EV,1.205846e+09,...,0,0-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,Stockholm-floor4
2,2-floor4_1-Sto,0,-2.020001,-0.080001,-2.015833,-0.073241,-1.800001,-0.200001,DW,1.205846e+09,...,1,0-floor4_1-Sto,3-floor4_1-Sto,4-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,Stockholm-floor4
3,3-floor4_1-Sto,0,-2.860001,0.499999,-3.019827,0.500790,-2.600001,0.599999,CR,1.205846e+09,...,0,2-floor4_1-Sto,4-floor4_1-Sto,6-floor4_1-Sto,9-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,Stockholm-floor4
4,4-floor4_1-Sto,0,-2.820001,-0.500001,-2.986465,-0.450972,-2.600001,-0.200001,CR,1.205846e+09,...,1,2-floor4_1-Sto,3-floor4_1-Sto,5-floor4_1-Sto,6-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,-1-floor4_1-Sto,Stockholm-floor4


In [3]:
node_number = 0
for i in data["node_id"]:
  # print(i)
  data = data.replace(i,node_number)
  node_number = node_number + 1

### Fixing "neighbor_id" according node_id

In [4]:
for i in data_list:
  for j in flist:
    for k in j:
      data.replace(str(-1)+"-"+k+"-"+i[:3],-1,inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   d1                 12009 non-null  float64
 5   d2                 12009 non-null  float64
 6   d3                 12009 non-null  float64
 7   d4                 12009 non-null  float64
 8   label              12009 non-null  object 
 9   d5                 12009 non-null  float64
 10  neighbor_ID_0      12009 non-null  int64  
 11  affordance_0       12009 non-null  float64
 12  view_num_0         12009 non-null  int64  
 13  neighbor_ID_1      12009 non-null  int64  
 14  affordance_1       12009 non-null  float64
 15  view_num_1         12009 non-null  int64  
 16  neighbor_ID_2      120

### Dropping out unnecessary columns

In [6]:
df = data.drop(['d1','d2','d3','d4','d5','neighbor_ID_0','neighbor_ID_1','neighbor_ID_2','neighbor_ID_3','neighbor_ID_4','neighbor_ID_5','neighbor_ID_6','neighbor_ID_7'], axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   label              12009 non-null  object 
 5   affordance_0       12009 non-null  float64
 6   view_num_0         12009 non-null  int64  
 7   affordance_1       12009 non-null  float64
 8   view_num_1         12009 non-null  int64  
 9   affordance_2       12009 non-null  float64
 10  view_num_2         12009 non-null  int64  
 11  affordance_3       12009 non-null  float64
 12  view_num_3         12009 non-null  int64  
 13  affordance_4       12009 non-null  float64
 14  view_num_4         12009 non-null  int64  
 15  affordance_5       12009 non-null  float64
 16  view_num_5         120

In [8]:
y = np.array(df['label'])
df['label'].unique().shape

(22,)

In [9]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.utils import to_categorical

In [10]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['locfloor']= label_encoder.fit_transform(df['locfloor'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   label              12009 non-null  object 
 5   affordance_0       12009 non-null  float64
 6   view_num_0         12009 non-null  int64  
 7   affordance_1       12009 non-null  float64
 8   view_num_1         12009 non-null  int64  
 9   affordance_2       12009 non-null  float64
 10  view_num_2         12009 non-null  int64  
 11  affordance_3       12009 non-null  float64
 12  view_num_3         12009 non-null  int64  
 13  affordance_4       12009 non-null  float64
 14  view_num_4         12009 non-null  int64  
 15  affordance_5       12009 non-null  float64
 16  view_num_5         120

In [11]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

In [12]:
labels = encode_onehot(np.array(df['label']))
labels.shape

(12009, 22)

In [13]:
df = df.drop(['label'],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009 entries, 0 to 12008
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   node_id            12009 non-null  int64  
 1   placeholder        12009 non-null  int64  
 2   x                  12009 non-null  float64
 3   y                  12009 non-null  float64
 4   affordance_0       12009 non-null  float64
 5   view_num_0         12009 non-null  int64  
 6   affordance_1       12009 non-null  float64
 7   view_num_1         12009 non-null  int64  
 8   affordance_2       12009 non-null  float64
 9   view_num_2         12009 non-null  int64  
 10  affordance_3       12009 non-null  float64
 11  view_num_3         12009 non-null  int64  
 12  affordance_4       12009 non-null  float64
 13  view_num_4         12009 non-null  int64  
 14  affordance_5       12009 non-null  float64
 15  view_num_5         12009 non-null  int64  
 16  affordance_6       120

In [14]:
def get_edge(data):
  edges = []
  if(data.new_neighbor_ID_0!=0):
    edges.append([data.node_id,data.new_neighbor_ID_0])
  else:
    pass
  if(data.new_neighbor_ID_1!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_1])
  else:
    pass
  if(data.new_neighbor_ID_2!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_2])
  else:
    pass
  if(data.new_neighbor_ID_3!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_3])
  else:
    pass
  if(data.new_neighbor_ID_4!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_4])
  else:
    pass
  if(data.new_neighbor_ID_5!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_5])
  else:
    pass
  if(data.new_neighbor_ID_6!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_6])
  else:
    pass
  if(data.new_neighbor_ID_7!=-1):
    edges.append([data.node_id,data.new_neighbor_ID_7])
  else:
    pass
  return edges

In [15]:
graph_edges = []
for i,raw_data in df.iterrows():
  graph_edges.extend(get_edge(raw_data))
edges_unordered = np.array(graph_edges)
edges_unordered.shape

(42783, 2)

In [16]:
# Initialize the adjacency matrix
A = np.zeros((12009, 12009))
A.shape

(12009, 12009)

In [17]:
for edge in graph_edges:
    A[int(edge[0]), int(edge[1])] = 1

# Print the adjacency matrix
print(A)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [18]:
!pip install spektral

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
X = np.array(df)
print(X.shape)
# X = np.reshape(X, (12009, 29, 1))
# Split the data into training and test sets
X_train, X_test,A_train,A_test, y_train, y_test = train_test_split(X,A,labels, test_size=0.2, random_state=42)
# Split the training data into training and validation sets
X_train, X_val,A_train,A_val, y_train, y_val = train_test_split(X_train,A_train, y_train, test_size=0.2, random_state=42)

(12009, 29)


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from spektral.layers import GCNConv

X_in = Input(shape=(X_train.shape[1],))
A_in = Input((None,), sparse=True)

X_1 = GCNConv(29, activation='relu')([X_in, A_in])
X_1 = Dropout(0.5)(X_1)
X_2 = GCNConv(16, activation='relu')([X_1, A_in])
X_2 = Dropout(0.5)(X_2)
output = Dense(22, activation='softmax')(X_2)

model = Model(inputs=[X_in, A_in], outputs=output)

In [21]:
optimizer = Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
train_data = X_train.reshape((-1,29))
A_train = A_train.reshape((-1, 29))
history = model.fit([X_train, A_train], y_train,
                    validation_split=0.1,
                    epochs=50, batch_size=64,
                    callbacks=[early_stopping])

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'model/gcn_conv/Relu' defined at (most recent call last):
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\jigar\AppData\Local\Programs\Python\Python37\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
 